In [1]:
!pip install cryptography
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.1 MB/s eta 0:00:00


In [22]:
import smtplib
import os
import base64
import imaplib
import email
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.fernet import Fernet
from cryptography.exceptions import InvalidSignature
from Crypto.PublicKey import DSA
from Crypto.Hash import SHA256

In [27]:
def generate_key(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        salt=salt,
        iterations=500000,
        length=32,
    )
    key = Fernet(base64.urlsafe_b64encode(kdf.derive(password)))
    return key

def encrypt_message(message, password, salt):
    key = generate_key(password, salt)

    # Converta a mensagem em bytes
    message_bytes = message.encode('utf-8')

    # Criptografe a mensagem
    encrypted_message = key.encrypt(message_bytes)

    return encrypted_message

def decrypt_message(encrypted_message, password, salt):
    key = generate_key(password, salt)

    # Descriptografe a mensagem
    decrypted_message_bytes = key.decrypt(encrypted_message)

    # Converta os bytes de volta para a mensagem original
    decrypted_message = decrypted_message_bytes.decode('utf-8')

    return decrypted_message

  # Função para gerar um par de chaves DSA
def generate_dsa_key_pair():
    key_size = 2048
    private_key = dsa.generate_private_key(key_size)
    public_key = private_key.public_key()
    return private_key, public_key

# Função para assinar uma mensagem com a chave privada DSA
def sign_message(private_key, message):
    message_bytes = message.encode('utf-8')
    signature = private_key.sign(message_bytes, hashes.SHA256())
    return signature

# Função para enviar a mensagem criptografada (opcional) e assinada por e-mail
def send_email(message, signature, encrypt=False):
    # Configurar as informações de email e senha
    from_email = 'wacbona@gmail.com'
    password = 'rega cguq dxyx cdsd'
    to_email = 'wacbona@gmail.com'

    # Configurar o servidor SMTP do Gmail
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587

    # Codificar a mensagem e a assinatura como base64
    message = base64.b64encode(message).decode('utf-8') if isinstance(message, bytes) else message
    signature = base64.b64encode(signature).decode('utf-8') if isinstance(signature, bytes) else signature

    # Criar o objeto MIMEMultipart para a mensagem
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = 'Assinatura Digital'

    # Se a opção de criptografia estiver ativada, criptografar a mensagem com AES
    if encrypt:
      # Criptografia com AES
      encrypted_message = encrypt_message(message, key, salt).decode('utf-8')

      msg.attach(MIMEText(encrypted_message, 'plain'))
    else:
      # Adicionar a mensagem ao corpo do email
      msg.attach(MIMEText(message, 'plain'))

    # Anexar a assinatura DSA ao email
    msg.attach(MIMEText(signature, 'plain'))

    # Iniciar a conexão SMTP e enviar o email
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(from_email, password)
        server.sendmail(from_email, to_email, msg.as_string())
        server.quit()
        print("Email enviado com sucesso!")
    except Exception as e:
        print(f"Erro ao enviar email: {str(e)}")


private_key, public_key = generate_dsa_key_pair()

#mensagem escolhida para ser enviada no email
message = "Esta é uma mensagem de teste!\n"

signature = sign_message(private_key, message)

salt = os.urandom(16)
key = b'segurancacomputacional'
encrypt = False

send_email(message, signature, encrypt)

Email enviado com sucesso!


In [28]:
# Configurar as informações de email e senha
email_address = 'wacbona@gmail.com'
password = 'rega cguq dxyx cdsd'

# Configurar o servidor IMAP do Gmail
imap_server = 'imap.gmail.com'
imap_port = 993

# Conectar-se ao servidor IMAP e ler os emails
try:
    mail = imaplib.IMAP4_SSL(imap_server)
    mail.login(email_address, password)
    mail.select("inbox")

    # Pesquisar por emails com o assunto "Mensagem Assinatura Digital"
    status, data = mail.search(None, 'SUBJECT "Assinatura Digital"')

    # Obter o ID do email mais recente encontrado
    email_ids = data[0].split()

    latest_email_id = email_ids[-1]

        # Recuperar o conteúdo do email mais recente
    status, msg_data = mail.fetch(latest_email_id, "(RFC822)")
    msg = email.message_from_bytes(msg_data[0][1])

        # Verificar se a mensagem possui partes anexadas (criptografia e assinatura)
    for idx, part in enumerate(msg.walk()):
            if part.get_content_type() == "text/plain":
                # A parte de texto contém a mensagem ou assinatura
                payload = part.get_payload(decode=True)
                if part.get_content_charset() is None:
                    charset = "utf-8"
                else:
                    charset = part.get_content_charset()

                # Decodificar a mensagem ou assinatura
                mensagem = payload.decode(charset)

                if encrypt and idx == 1:
                  mensagem = decrypt_message(mensagem, key, salt)

                if idx == 1:
                  if verify_signature(public_key, mensagem, signature):
                    print("Assinatura Válida!\n")
                  else:
                    print("Assinatura inválida!\n")

                print(mensagem)
except Exception as e:
    print(f"Erro ao ler email: {str(e)}")

Assinatura Válida!

Esta é uma mensagem de teste!

MEQCIF93+cTWF9qKGnUO4AsUpo/SLUgpdwXYTFC4IgCAn2fsAiBsda8MIIB0fYUTswiV1u6XZ8hs0P/9+3aRPt2gNup2tw==
